In [8]:
import requests 
import json
import sys
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pytesseract
import numpy as np
import pandas as pd
import datetime
from PIL import Image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [9]:
def time_msec(time):
    time_split=time.split(':')
    if len(time_split) == 2 :
        sec = int(time_split[0])*60+int(time_split[1])
        return sec*1000
    elif len(time_split) == 3 :
        sec = (int(time_split[0])*60+int(time_split[1]))*60+int(time_split[2])
        return sec*1000

def msec_time(msec):
    sec = msec/1000
    return str(int(sec//60))+':'+str(int(sec%60))

# 1-1. 트위치 채팅 내역 수집

In [210]:
def doubleDigit(num):
    if num < 10 :
        return '0'+str(num)
    else:
        return str(num)

def twichchat(videoId,clientId):
#     videoId = '571178493'
#     clientId = 'vhggq5l1jefc8dtfza2phfc6ih69gs'
    
    videoId = videoId
    clientId = clientId
    
    chat = []
    time = []
    user = []

    nextCursor = ''

    params = {}
    params['client_id'] = clientId

    i = 0
    while True :

        if i == 0 :
            URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?content_offset_seconds=0' 
            i += 1
        else:
            URL = 'https://api.twitch.tv/v5/videos/'+videoId+'/comments?cursor=' 
            URL +=  nextCursor   


        response = requests.get(URL, params=params)

        j = json.loads(response.text)

        for k in range(0,len(j["comments"])):
            timer = j["comments"][k]["content_offset_seconds"]

            timeMinute = int(timer/60)

            if timeMinute >= 60 :
                timeHour = int(timeMinute/60)
                timeMinute %= 60
            else :
                timeHour = int(timeMinute/60)

            timeSec = int(timer%60)

            time.append(doubleDigit(timeHour)+':'+doubleDigit(timeMinute)+':'+doubleDigit(timeSec))
            user.append(j["comments"][k]["commenter"]["display_name"])
            chat.append(j["comments"][k]["message"]["body"])
        if '_next' not in j:
            break

        nextCursor = j["_next"]
    return time

    f = open(videoId+".txt", 'wt', encoding="utf-8")

    for x in range(0, len(time)): 
            f.write('[')
            f.write(str(time[x]))
            f.write(']')
            f.write(' ')
            f.write('<')
            f.write(str(user[x]))
            f.write('>')
            f.write(' ')
            f.write(str(chat[x]))
            f.write("\n")

    f.close()

In [284]:
timedata = twichchat('571178493', 'vhggq5l1jefc8dtfza2phfc6ih69gs')

In [285]:
time = timedata[:]

In [306]:
time

['0:00:04',
 '0:00:05',
 '0:00:07',
 '0:00:07',
 '0:00:08',
 '0:00:09',
 '0:00:12',
 '0:00:19',
 '0:00:19',
 '0:00:21',
 '0:00:34',
 '0:00:34',
 '0:00:39',
 '0:00:41',
 '0:00:42',
 '0:00:45',
 '0:00:47',
 '0:00:47',
 '0:00:48',
 '0:00:49',
 '0:00:51',
 '0:00:52',
 '0:00:55',
 '0:00:56',
 '0:00:58',
 '0:01:01',
 '0:01:02',
 '0:01:02',
 '0:01:03',
 '0:01:06',
 '0:01:06',
 '0:01:07',
 '0:01:08',
 '0:01:08',
 '0:01:09',
 '0:01:11',
 '0:01:12',
 '0:01:12',
 '0:01:13',
 '0:01:14',
 '0:01:14',
 '0:01:14',
 '0:01:15',
 '0:01:15',
 '0:01:16',
 '0:01:16',
 '0:01:16',
 '0:01:18',
 '0:01:19',
 '0:01:19',
 '0:01:20',
 '0:01:20',
 '0:01:20',
 '0:01:21',
 '0:01:21',
 '0:01:22',
 '0:01:22',
 '0:01:22',
 '0:01:22',
 '0:01:23',
 '0:01:24',
 '0:01:26',
 '0:01:26',
 '0:01:26',
 '0:01:26',
 '0:01:28',
 '0:01:28',
 '0:01:28',
 '0:01:29',
 '0:01:29',
 '0:01:30',
 '0:01:30',
 '0:01:30',
 '0:01:31',
 '0:01:32',
 '0:01:33',
 '0:01:33',
 '0:01:33',
 '0:01:33',
 '0:01:34',
 '0:01:34',
 '0:01:37',
 '0:01:37',
 '0:

In [286]:
for i in range(len(time)):
    time[i] = time[i][1:]

# 1-2. OCR을 이용해 영상에서 정보 추출

In [40]:
def ocr1(frame):
#     스코어 ocr용 함수
    board=frame[:25,1550:1905]
    width = board.shape[1]
    score=board[:,5:int(width*0.21)]

    sharpening = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    sharp = cv2.filter2D(score, -1, sharpening)

    fx=4
    fy=4
    resize=cv2.resize(sharp, dsize=(0,0), fx=fx,fy=fy)

    gray = cv2.cvtColor(resize, cv2.COLOR_RGB2GRAY)

    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    _, th = cv2.threshold(blur, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    
    str_score = [pytesseract.image_to_string(th)]
    return str_score

In [41]:
def ocr2(frame):
#     kda, 시간 ocr 함수
    board=frame[:25,1550:1905]
    gray = cv2.cvtColor(board, cv2.COLOR_RGB2GRAY)

    fx=4
    fy=4

    board=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)

    img_blur = cv2.GaussianBlur(board, (3,3), 0)
    _, board = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    width = board.shape[1]

    kda = board[:,int(width*0.32):int(width*0.48)]
    
    data = []
    str_kda = pytesseract.image_to_string(kda)
    data.append(str_kda)
    
    time = board[:,int(width*0.87):]
    str_time = pytesseract.image_to_string(time)
    data.append(str_time)

    return data

In [42]:
def ocr3(frame):
#     골드 ocr 함수
    gold=frame[1047:1070,1180:1237]
    gray = cv2.cvtColor(gold, cv2.COLOR_RGB2GRAY)
    
    fx=4
    fy=4
    resize=cv2.resize(gray, dsize=(0,0), fx=fx,fy=fy)
    blur = cv2.GaussianBlur(resize, (3, 3), 0)
    _, th = cv2.threshold(blur, 0, 255,  cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)

    str_gold = [pytesseract.image_to_string(th)]
    return str_gold

In [283]:
# 동영상 프로세싱
cap = cv2.VideoCapture('video/full.mp4')

starttime = '2:06:38'
endtime = '2:22:11'

timemsec = time_msec(starttime)
cap.set(cv2.CAP_PROP_POS_MSEC, timemsec)

i=0
datalist=[]
while (cap.isOpened):
    i += 1
    ret, frame = cap.read()
    if ret:
        score = ocr1(frame)
        kda_time = ocr2(frame)
        gold = ocr3(frame)
        vid_time = [cap.get(cv2.CAP_PROP_POS_MSEC)]
        data = score + kda_time + gold + vid_time
        datalist.append(data)
        if i%60 == 0:
            print(data)
        
        timemsec += 1000
        cap.set(cv2.CAP_PROP_POS_MSEC, timemsec)
        
        if vid_time[0] > time_msec(endtime):
            break
    else:
        break

['O vsQ', '0/0/0', '01:10', '', 7657001.0]
['0 vsO', '0/0/0', '02:10', '', 7717001.0]
['Ovs2', '2/0/0', '03:10', '1217', 7777001.0]
['', '2/0/0', '04:10', '1264', 7837001.0]
['T vs4', '3/0/0', '05:10', '1835', 7897001.0]
['Ol', '4/0/0', '06:10', '707', 7957001.0]
['2Z2vs6', '5/0/0', '07%', '39', 8017001.0]
['Avs/', '5/0/0', '08:10', '415', 8077001.0]
['Avs /', '5/0/0', '09:10', '875', 8137001.0]
['Avs8', '6/0/0', '10:10', '605', 8197001.0]
['4vs8', '6/0/0', '11:10', '415', 8257001.0]
['5 vs 13', '6/1/2', '12:10', '707', 8317001.0]
['5 vs 15', '7/1/3', '13:10', '1345', 8377001.0]
['Bvs19', '8/1/4', '14:10', '1186', 8437001.0]
['5 vs 19', '8/1/4', '15:10', '1923', 8497001.0]


# 2-1. 수집된 데이터 가공(필터링)

In [287]:
df=pd.DataFrame(datalist)
df

,0,1,2,3,4
0,"(, vs",0/0/0,00:11,500,7598000.0
1,Avs0,0/0/0,00:12,,7599001.0
2,0 vsO,0/0/0,00:13,,7600000.0
3,O vsO,0/0/0,00:14,,7601001.0
4,Ovsd,0/0/0,00:15,,7602000.0
...,...,...,...,...,...
929,Bvs22,8/1/5,15:40,2209,8527001.0
930,Bvs22,8/1/5,15:41,2211,8528000.0
931,Bvs22,8/1/5,15:42,2213,8529001.0
932,5 vs 22,8/1/5,15:43,2215,8530000.0


In [288]:
# kda 제대로 출력되었는지 확인. 세 값이 모두 있지 않을경우 Na처리.
for i in range(len(df)):
    if len(df[1][i].split('/'))!=3:
        df[1][i]=np.nan
df

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,0,1,2,3,4
0,"(, vs",0/0/0,00:11,500,7598000.0
1,Avs0,0/0/0,00:12,,7599001.0
2,0 vsO,0/0/0,00:13,,7600000.0
3,O vsO,0/0/0,00:14,,7601001.0
4,Ovsd,0/0/0,00:15,,7602000.0
...,...,...,...,...,...
929,Bvs22,8/1/5,15:40,2209,8527001.0
930,Bvs22,8/1/5,15:41,2211,8528000.0
931,Bvs22,8/1/5,15:42,2213,8529001.0
932,5 vs 22,8/1/5,15:43,2215,8530000.0


In [289]:
# score를 vs 기준으로 나누고, kda를 / 기준으로 나눠서 새로운 col 생성
df['fr']=df[0].str.split('vs').str[0]
df['en']=df[0].str.split('vs').str[1]
df['k']=df[1].str.split('/').str[0]
df['d']=df[1].str.split('/').str[1]
df['a']=df[1].str.split('/').str[2]

In [290]:
# object 타입을 숫자로 변환. 숫자가 아닐시 강제로(Na)
df['fr']=pd.to_numeric(df['fr'], errors='coerce')
df['en']=pd.to_numeric(df['en'], errors='coerce')
df['k']=pd.to_numeric(df['k'], errors='coerce')
df['d']=pd.to_numeric(df['d'], errors='coerce')
df['a']=pd.to_numeric(df['a'], errors='coerce')
df[3]=pd.to_numeric(df[3], errors='coerce')

In [291]:
# 시간을 분:초 형식으로 바꾸는건데. 안해도 될것같음.
df['time']=df[4].map(lambda x : str(datetime.timedelta(milliseconds = (x//1000)*1000)))
df

,0,1,2,3,4,fr,en,k,d,a,time
0,"(, vs",0/0/0,00:11,500.0,7598000.0,NaN,NaN,0.0,0.0,0.0,2:06:38
1,Avs0,0/0/0,00:12,NaN,7599001.0,NaN,0.0,0.0,0.0,0.0,2:06:39
2,0 vsO,0/0/0,00:13,NaN,7600000.0,0.0,NaN,0.0,0.0,0.0,2:06:40
3,O vsO,0/0/0,00:14,NaN,7601001.0,NaN,NaN,0.0,0.0,0.0,2:06:41
4,Ovsd,0/0/0,00:15,NaN,7602000.0,NaN,NaN,0.0,0.0,0.0,2:06:42
...,...,...,...,...,...,...,...,...,...,...,...
929,Bvs22,8/1/5,15:40,2209.0,8527001.0,NaN,22.0,8.0,1.0,5.0,2:22:07
930,Bvs22,8/1/5,15:41,2211.0,8528000.0,NaN,22.0,8.0,1.0,5.0,2:22:08
931,Bvs22,8/1/5,15:42,2213.0,8529001.0,NaN,22.0,8.0,1.0,5.0,2:22:09
932,5 vs 22,8/1/5,15:43,2215.0,8530000.0,5.0,22.0,8.0,1.0,5.0,2:22:10


In [292]:
# 시간을 인덱스로 사용
df.set_index('time', inplace=True)
df

,0,1,2,3,4,fr,en,k,d,a
time,,,,,,,,,,
2:06:38,"(, vs",0/0/0,00:11,500.0,7598000.0,NaN,NaN,0.0,0.0,0.0
2:06:39,Avs0,0/0/0,00:12,NaN,7599001.0,NaN,0.0,0.0,0.0,0.0
2:06:40,0 vsO,0/0/0,00:13,NaN,7600000.0,0.0,NaN,0.0,0.0,0.0
2:06:41,O vsO,0/0/0,00:14,NaN,7601001.0,NaN,NaN,0.0,0.0,0.0
2:06:42,Ovsd,0/0/0,00:15,NaN,7602000.0,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2:22:07,Bvs22,8/1/5,15:40,2209.0,8527001.0,NaN,22.0,8.0,1.0,5.0
2:22:08,Bvs22,8/1/5,15:41,2211.0,8528000.0,NaN,22.0,8.0,1.0,5.0
2:22:09,Bvs22,8/1/5,15:42,2213.0,8529001.0,NaN,22.0,8.0,1.0,5.0


In [305]:
df3.to_excel('df3.xlsx')

In [294]:
# 채팅 dataframe 생성. 인덱스를 타임으로
timeseries = pd.Series(time)
df2 = pd.DataFrame(index=timeseries.unique())
df2['count'] = timeseries.value_counts()

df2.index.name = 'time'

In [295]:
df2

,count
time,
0:00:04,1
0:00:05,1
0:00:07,2
0:00:08,1
0:00:09,1
...,...
2:23:58,3
2:23:59,3
2:24:00,7


In [296]:
# 시간 열을 기준으로 데이터 합치기
df3 = pd.merge(df,df2,on='time')

In [297]:
df3

,0,1,2,3,4,fr,en,k,d,a,count
time,,,,,,,,,,,
2:06:38,"(, vs",0/0/0,00:11,500.0,7598000.0,NaN,NaN,0.0,0.0,0.0,2
2:06:39,Avs0,0/0/0,00:12,NaN,7599001.0,NaN,0.0,0.0,0.0,0.0,4
2:06:40,0 vsO,0/0/0,00:13,NaN,7600000.0,0.0,NaN,0.0,0.0,0.0,2
2:06:41,O vsO,0/0/0,00:14,NaN,7601001.0,NaN,NaN,0.0,0.0,0.0,1
2:06:42,Ovsd,0/0/0,00:15,NaN,7602000.0,NaN,NaN,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2:22:07,Bvs22,8/1/5,15:40,2209.0,8527001.0,NaN,22.0,8.0,1.0,5.0,3
2:22:08,Bvs22,8/1/5,15:41,2211.0,8528000.0,NaN,22.0,8.0,1.0,5.0,1
2:22:09,Bvs22,8/1/5,15:42,2213.0,8529001.0,NaN,22.0,8.0,1.0,5.0,1


In [298]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 830 entries, 2:06:38 to 2:22:11
Data columns (total 11 columns):
0        830 non-null object
1        814 non-null object
2        830 non-null object
3        695 non-null float64
4        830 non-null float64
fr       279 non-null float64
en       482 non-null float64
k        814 non-null float64
d        813 non-null float64
a        814 non-null float64
count    830 non-null int64
dtypes: float64(7), int64(1), object(3)
memory usage: 77.8+ KB


# 2-2. Feature Engineering

<img src='fea.jpg'>

In [299]:
df3['d_kda']=pd.Series()
df3['d_gold'] = pd.Series()
df3['n_chat']=pd.Series()

# for i in reversed(range(len(df3))):
#     df3['d_kda'][i] = df3['kda'][i]-df3['kda'][i-15]

for i in range(len(df3)):
    df3['d_gold'][i] = df3[3][i] - df3[3][i-15]

# D_K D_D D_A 컬럼 추가. [i]-[i-15]
df3['d_k']=pd.Series()
for i in reversed(range(len(df3))):
    df3['d_k'][i] = df3['k'][i]-df3['k'][i-15]
df3['d_d']=pd.Series()
for i in reversed(range(len(df3))):
    df3['d_d'][i] = df3['d'][i]-df3['d'][i-15] 
df3['d_a']=pd.Series()
for i in reversed(range(len(df3))):
    df3['d_a'][i] = df3['a'][i]-df3['a'][i-15]
    
# 채팅 1~15값 더하기
for i in range(len(df3)):
    df3['n_chat'][i] = sum(df3['count'][i:i+15])
    
# d_kda 컬럼 전처리. 음수=0, 5이상=0
df3['d_kda'][df3['d_kda']>5] = 0
df3['d_kda'][df3['d_kda']<0] = 0
    
# 5이상과 음수값 없애기
df3['d_k'][df3['d_k']>5] = 0
df3['d_k'][df3['d_k']<0] = 0
df3['d_d'][df3['d_d']>5] = 0
df3['d_d'][df3['d_d']<0] = 0
df3['d_a'][df3['d_a']>5] = 0
df3['d_a'][df3['d_a']<0] = 0

df3['d_gold'][df3['d_gold']<0] = np.nan
df3['d_gold'][df3['d_gold']>2000] = np.nan    

df3['d_kda'] = df3['d_k']+df3['d_d']+df3['d_a']

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be se

In [300]:
df3.head()

,0,1,2,3,4,fr,en,k,d,a,count,d_kda,d_gold,n_chat,d_k,d_d,d_a
time,,,,,,,,,,,,,,,,,
2:06:38,"(, vs",0/0/0,00:11,500.0,7598000.0,NaN,NaN,0.0,0.0,0.0,2,0.0,NaN,35.0,0.0,0.0,0.0
2:06:39,Avs0,0/0/0,00:12,NaN,7599001.0,NaN,0.0,0.0,0.0,0.0,4,0.0,NaN,36.0,0.0,0.0,0.0
2:06:40,0 vsO,0/0/0,00:13,NaN,7600000.0,0.0,NaN,0.0,0.0,0.0,2,0.0,NaN,33.0,0.0,0.0,0.0
2:06:41,O vsO,0/0/0,00:14,NaN,7601001.0,NaN,NaN,0.0,0.0,0.0,1,0.0,NaN,33.0,0.0,0.0,0.0
2:06:42,Ovsd,0/0/0,00:15,NaN,7602000.0,NaN,NaN,0.0,0.0,0.0,1,0.0,NaN,34.0,0.0,0.0,0.0


# 2-3. 최종 데이터셋 생성

In [301]:
# 데이터셋 제작.
dataset = df3[['d_kda','d_k','d_d','d_a','d_gold','n_chat']][15:-15]

# 라벨값(누적 채팅수)를 일정 수치를 기준으로 0과 1로 나눔. sigmoid 사용 위해
line = dataset['n_chat'].mean()+dataset['n_chat'].std()
for i in range(len(dataset)):
    if dataset['n_chat'][i] > line :
        dataset['n_chat'][i] = 1
    else :
        dataset['n_chat'][i] = 0
        
# 결측치 제거        
dataset.dropna(axis=0, inplace=True)

# 골드 변화량 수치를 0~1사이의 값으로 줄임
dataset['d_gold'] = dataset['d_gold']/dataset['d_gold'].max()

In [302]:
dataset

,d_kda,d_k,d_d,d_a,d_gold,n_chat
time,,,,,,
2:08:30,0.0,0.0,0.0,0.0,0.026794,0.0
2:08:31,0.0,0.0,0.0,0.0,0.028708,0.0
2:08:36,0.0,0.0,0.0,0.0,0.034450,0.0
2:08:39,0.0,0.0,0.0,0.0,0.056459,0.0
2:08:43,0.0,0.0,0.0,0.0,0.075598,0.0
...,...,...,...,...,...,...
2:21:52,1.0,0.0,0.0,1.0,0.144498,1.0
2:21:53,1.0,0.0,0.0,1.0,0.164593,1.0
2:21:54,1.0,0.0,0.0,1.0,0.163636,1.0


In [303]:
dataset.to_excel('dataset.xlsx')